Let's try to get tuples to work with range https://github.com/numba/numba/issues/2771

We can do this with recursion

I wanna be able to construct a tuple from this signature:

`create_tuple(lambda i: ..., n)`

So that:

```python
create_tuple(lambda i: i**2, 3) == (0, 1, 4)
```

In [1]:
from numba import njit

In [3]:
@njit
def create_tuple(f, n, i=0):
    if i == n - 1:
        return (f(i),)
    return (f(i),) + create_tuple(f, n, i + 1)


@njit
def f(i):
    return i**2

@njit
def test():
    return create_tuple(f, 3)

test()

TypingError: Failed at nopython (nopython frontend)
Failed at nopython (nopython frontend)
Failed at nopython (nopython frontend)
Can't unify return type from the following types: tuple(int64 x 1), tuple(int64 x 2)
Return of: IR name '$12.5', type 'tuple(int64 x 1)', location: 
File "<ipython-input-3-203536fd220c>", line 4:
def create_tuple(f, n, i=0):
    <source elided>
    if i == n - 1:
        return (f(i),)
        ^
Return of: IR name '$22.13', type 'tuple(int64 x 2)', location: 
File "<ipython-input-3-203536fd220c>", line 5:
def create_tuple(f, n, i=0):
    <source elided>
        return (f(i),)
    return (f(i),) + create_tuple(f, n, i + 1)
    ^
[1] During: resolving callee type: recursive(type(CPUDispatcher(<function create_tuple at 0x113e94598>)))
[2] During: typing of call at <ipython-input-3-203536fd220c> (5)


File "<ipython-input-3-203536fd220c>", line 5:
def create_tuple(f, n, i=0):
    <source elided>
        return (f(i),)
    return (f(i),) + create_tuple(f, n, i + 

OK that doesn't work... Instead we create $n$ dispatchers, one for each tuple size.

In [54]:
def create_tuple_creator(f, n):
    assert n > 0
    f = njit(f)
    @njit
    def creator(args):
        return (f(0, *args),)
    for i in range(1, n):
        # need to pass in creator and i to lambda to capture in scope
        @njit
        def creator(args, creator=creator, i=i):
            return creator(args) + (f(i, *args),)
    return njit(lambda *args: creator(args))

creator = create_tuple_creator(lambda i, j: i**j, 4)

@njit
def test():
    return creator(3), creator(2)

test()

((0, 1, 8, 27), (0, 1, 4, 9))

Now let's re-create some of the examples in the numba issue: https://github.com/numba/numba/issues/2771#issuecomment-368620310

In [55]:
range_10 = create_tuple_creator(lambda i: i, 10)
range_10_x = create_tuple_creator(lambda i, x: i*x, 10)
true_10 = create_tuple_creator(lambda _: True, 10)
false_10 = create_tuple_creator(lambda _: False, 10)
zip_10 = create_tuple_creator(lambda i, l, r: (l[i], r[i]), 10)

@njit
def foo(x):
    print(range_10()) # tuple([i for i in range(10)])
    print(range_10_x(x)) # tuple([i*x for i in range(10)])
    print(zip_10(true_10(), false_10())),  # tuple(zip((True,)*10, (False,)*10))

    
foo(2)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)
((True, False), (True, False), (True, False), (True, False), (True, False), (True, False), (True, False), (True, False), (True, False), (True, False))
